In [1]:
from quspin.operators import hamiltonian, commutator 
from scipy.sparse import csr_matrix as csr_array
from quspin.basis import boson_basis_1d
import numpy as np 

class boson_basis_1d_updated(boson_basis_1d):
    def int_to_fock(self, state):
        if int(state) != state:
            raise ValueError("state must be integer")
        bits = np.array([int(state) // int(self.sps**(self.N - i - 1)) % self.sps for i in range(self.N)])
        return bits

    def fock_to_index(self, fock_repr):
        fock_str = "".join([str(bit) for bit in fock_repr])
        return self._index(fock_str)

#Constructs the theta dependent part of H
def Hdyn(J,theta):

    indptr = [0]
    indices = []
    data = []
    
    for j in range(basis.Ns):
        ind_ctr = 0
        state = basis.states[j]
        fock_state = basis.int_to_fock(state)
        lattice_edge = L if PBC else L - 1
        
        for i in range(lattice_edge):
            # Forward hopping
            if fock_state[i] < basis.sps - 1 and fock_state[(i+1)%L] > 0:
                new_state = fock_state.copy()
                new_state[i] += 1
                new_state[(i+1)%L] -= 1 

                indices.append(basis.fock_to_index(new_state))
                data.append(-J * np.sqrt(new_state[i]) * np.sqrt(fock_state[(i+1)%L]) * np.exp(1j * theta * fock_state[i]))
                ind_ctr += 1
            
            # Backward hopping
            if fock_state[(i+1)%L] < basis.sps - 1 and fock_state[i] > 0:
                new_state = fock_state.copy()
                new_state[i] -= 1
                new_state[(i+1)%L] += 1
    
                indices.append(basis.fock_to_index(new_state))
                data.append(-J * np.sqrt(new_state[(i+1)%L]) * np.sqrt(fock_state[i]) * np.exp(-1j * theta * new_state[i]))
                ind_ctr += 1
    
        indptr.append(indptr[-1] + ind_ctr)
    
    H_hop = csr_array((np.array(data), np.array(indices), np.array(indptr)), shape=(basis.Ns, basis.Ns))

    return H_hop




#Constructs the full H as function of theta
def H_theta(J, theta, U):

    
    OS_int = [[0.5 * U, i, i] for i in range(L)]

    # Construct Remaining Hamiltonian Terms
    static = [['nn', OS_int]]
    Hstat = hamiltonian(static, [], basis=basis, dtype=np.complex128, check_herm=False, check_symm=False, check_pcon=False);

    
    return Hstat + Hdyn(J,theta)


In [5]:
L=8
Nb=4
PBC=False
basis = boson_basis_1d_updated(L, Nb=Nb)
H_theta(1.0, 0.7, 0.5)

<quspin.operators.hamiltonian:
static mat: <Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 2010 stored elements and shape (330, 330)>
dynamic:>